In [ ]:
import pandas as pd
from pathlib import Path

# ── 1.  Load Silver files ────────────────────────────────────────────
pollutant_path = Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/pollutant/pollutant_hourly_2025.parquet")
weather_path   = Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/met/weather_hourly_2025.parquet")

poll = pd.read_parquet(pollutant_path)            # 896 k rows
wx   = pd.read_parquet(weather_path)              # 44 k rows

# ── 2.  Deduplicate Met-Office rows ───────────── (latest run per hour)
wx = (
    wx.sort_values(["site_id", "timestamp", "forecast_run"])
      .drop_duplicates(subset=["site_id", "timestamp"], keep="last")
)

# ── 3.  Keep only forecasts ISSUED before or at the hour predicted ──
wx = wx[wx["forecast_run"] <= wx["timestamp"]]

# ── 4.  Build lag (+1 h target) in pollutant table ──────────────────
poll = poll.sort_values(["site_id", "date_time"])
grp  = poll.groupby("site_id")

poll["pm25_t-1"]   = grp["pm25"].shift(1)
poll["no2_t-1"]    = grp["no2"].shift(1)
poll["o3_t-1"]     = grp["o3"].shift(1)

poll["pm25_target"] = grp["pm25"].shift(-1)
poll["no2_target"]  = grp["no2"].shift(-1)
poll["o3_target"]   = grp["o3"].shift(-1)

# ── 5.  Join on site_id + hour (timestamp) ──────────────────────────
joined = (
    poll.merge(
        wx, left_on=["site_id", "date_time"],
            right_on=["site_id", "timestamp"],
            how="left"
    )
    .drop(columns=["timestamp"])                 # keep one time column
)

# after building `joined` DataFrame:
joined = joined.rename(columns={
    "pm25_t-1": "pm25_t_1",
    "no2_t-1":  "no2_t_1",
    "o3_t-1":   "o3_t_1",
})
joined.to_parquet("data/silver/joined/hourly_joined_2025_from_28jul.parquet", index=False)


out_path = Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/joined/hourly_joined_2025.parquet")
out_path.parent.mkdir(parents=True, exist_ok=True)
joined.to_parquet(out_path, index=False)

print("Joined rows:", len(joined))
print("Sample:", joined.head(3))


Joined rows: 896384
Sample:   site_id                 date_time  pm25       no2  o3  pm25_t-1   no2_t-1  \
0    ABD7 2025-01-01 01:00:00+00:00   NaN  15.10875 NaN       NaN       NaN   
1    ABD7 2025-01-01 02:00:00+00:00   NaN  43.60500 NaN       NaN  15.10875   
2    ABD7 2025-01-01 03:00:00+00:00   NaN  29.45250 NaN       NaN  43.60500   

   o3_t-1  pm25_target  no2_target  o3_target forecast_run  temp  wind  \
0     NaN          NaN     43.6050        NaN          NaT   NaN   NaN   
1     NaN          NaN     29.4525        NaN          NaT   NaN   NaN   
2     NaN          NaN     29.0700        NaN          NaT   NaN   NaN   

   humidity  
0       NaN  
1       NaN  
2       NaN  


There are some Limitations in the data extracted. However, I am limiting the data to have high quality ml training. As the ingestion pipeline keeps updating, the data will keep getting enriched.

In [11]:
# ── paths ───────────────────────────────────────────────────────────
JOINED_IN  = Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/joined/hourly_joined_2025.parquet")
JOINED_OUT = Path("/workspaces/airoute_mlops/airoute_mlops/data/silver/joined/hourly_joined_2025_from_28jul.parquet")
JOINED_OUT.parent.mkdir(parents=True, exist_ok=True)

# ── load, filter, save ──────────────────────────────────────────────
df = pd.read_parquet(JOINED_IN)

cutoff = pd.Timestamp("2025-07-28 00:00", tz="UTC")
df_filtered = df[df["date_time"] >= cutoff]

df_filtered.to_parquet(JOINED_OUT, index=False)
print("Saved:", JOINED_OUT)
print("Rows kept:", len(df_filtered))


Saved: /workspaces/airoute_mlops/airoute_mlops/data/silver/joined/hourly_joined_2025_from_28jul.parquet
Rows kept: 20447


In [12]:
data = pd.read_parquet(JOINED_OUT)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20447 entries, 0 to 20446
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype              
---  ------        --------------  -----              
 0   site_id       20447 non-null  object             
 1   date_time     20447 non-null  datetime64[ns, UTC]
 2   pm25          15799 non-null  float64            
 3   no2           15613 non-null  float64            
 4   o3            9863 non-null   float64            
 5   pm25_t-1      15800 non-null  float64            
 6   no2_t-1       15614 non-null  float64            
 7   o3_t-1        9862 non-null   float64            
 8   pm25_target   15797 non-null  float64            
 9   no2_target    15614 non-null  float64            
 10  o3_target     9866 non-null   float64            
 11  forecast_run  9589 non-null   datetime64[ns, UTC]
 12  temp          9589 non-null   float64            
 13  wind          9589 non-null   float64            
 14  humidi

In [14]:
data.tail()

,site_id,date_time,pm25,no2,o3,pm25_t-1,no2_t-1,o3_t-1,pm25_target,no2_target,o3_target,forecast_run,temp,wind,humidity
20442,YW,2025-08-01 19:00:00+00:00,2.170,1.33875,62.26584,2.264,1.14750,61.46756,1.792,1.14750,57.67573,2025-08-01 17:00:00+00:00,16.49,4.08,61.63
20443,YW,2025-08-01 20:00:00+00:00,1.792,1.14750,57.67573,2.170,1.33875,62.26584,1.604,1.72125,53.28519,2025-08-01 17:00:00+00:00,14.83,3.54,65.69
20444,YW,2025-08-01 21:00:00+00:00,1.604,1.72125,53.28519,1.792,1.14750,57.67573,1.509,1.33875,51.68863,2025-08-01 17:00:00+00:00,13.57,3.15,70.98
20445,YW,2025-08-01 22:00:00+00:00,1.509,1.33875,51.68863,1.604,1.72125,53.28519,1.604,1.33875,51.48906,2025-08-01 17:00:00+00:00,12.79,3.12,75.23
20446,YW,2025-08-01 23:00:00+00:00,1.604,1.33875,51.48906,1.509,1.33875,51.68863,1.887,0.62967,56.52820,2025-08-01 17:00:00+00:00,12.40,3.17,78.82
